In [6]:
!pip install matplotlib
!pip install lightning timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 4.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01


In [1]:

import pandas as pd
import os
import multiprocessing as mp

# Then import other libraries and define your code
import random
from collections import defaultdict, Counter

import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns

from sklearn.metrics import confusion_matrix, average_precision_score
from sklearn.metrics import multilabel_confusion_matrix as ConfusionMatrix
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, Subset
from transformers import get_cosine_schedule_with_warmup
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torchvision.transforms import AutoAugment, AutoAugmentPolicy

import lightning.pytorch as pl
from lightning.pytorch import Trainer
from lightning.pytorch.strategies import DDPStrategy
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

from torchmetrics.classification import Accuracy, Precision, Recall, F1Score, MulticlassF1Score
from torchmetrics.classification import BinaryPrecisionRecallCurve

#from bayes_opt import BayesianOptimization
from tqdm import tqdm
import timm

from sklearn.metrics import precision_recall_curve
from torchvision.models import resnet50, ResNet50_Weights



/Users/beszabo/Documents/meme/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(in_channels, in_channels // reduction, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(in_channels // reduction, in_channels, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.global_avg_pool(x).view(b, c)
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y).view(b, c, 1, 1)
        return x * y


In [3]:
class ResNetWrapper(pl.LightningModule):
    def __init__(self, lr, dropout_rate=0.5, ood_threshold=0.5):
        super().__init__()
        self.lr = lr
        self.ood_threshold = ood_threshold
        self.dropout_rate = dropout_rate
        self.register_buffer('sigmoid_thresholds', torch.ones(80, dtype=torch.float32) * 0.5)  # Initial 0.5 for all classes
        self.validation_step_y_hats = []

        # Data collection for threshold tuning
        self.validation_logits = []
        self.validation_targets = []

        # testing
        self.validation_step_ys = []

        backbone = timm.create_model('seresnet152d', pretrained=True, num_classes=80)
        layers = list(backbone.children())[:-1]
        layers.insert(-1, SEBlock(in_channels=2048))  # Insert FIRST
        self.feature_extractor = nn.Sequential(*layers)  # Then create
        # In __init__
        assert isinstance(self.feature_extractor[-2], SEBlock), "SEBlock not in model!"

        for param in self.feature_extractor.parameters():
            param.requires_grad = False

        layers.insert(-1, SEBlock(in_channels=2048))  # 2048 az utolsó réteg csatornáinak száma ResNet50 esetén

        # Binary classifier head for OOD detection
        self.ood_classifier = nn.Sequential(
            nn.Dropout(self.dropout_rate),
            nn.Linear(backbone.fc.in_features, 1)  # Single output for binary classification
        )

        self.classifier = nn.Sequential(
            nn.Dropout(self.dropout_rate),  # hyperparameter
            nn.Linear(backbone.fc.in_features, out_features=80)
        )

        # Separate loss functions for both tasks
        self.loss_fn_class = nn.BCEWithLogitsLoss()  # Multi-label loss
        self.loss_fn_ood = nn.BCEWithLogitsLoss()    # Binary OOD loss

        self.train_acc = Accuracy(task="multilabel", num_labels=80)
        self.val_acc = Accuracy(task="multilabel", num_labels=80)
        self.train_precision = Precision(task="multilabel", num_labels=80)
        self.val_precision = Precision(task="multilabel", num_labels=80)
        self.train_recall = Recall(task="multilabel", num_labels=80)
        self.val_recall = Recall(task="multilabel", num_labels=80)
        self.train_f1 = F1Score(task="multilabel", num_labels=80)
        self.val_f1 = F1Score(task="multilabel", num_labels=80)
        self.ood_acc = Accuracy(task="binary") 
        self.val_acc_id = Accuracy(task="multilabel", num_labels=80)
        self.val_f1_id = F1Score(task="multilabel", num_labels=80)
        self.val_precision_id = Precision(task="multilabel", num_labels=80)
        self.val_recall_id = Recall(task="multilabel", num_labels=80)


    def forward(self, x):
        # Simple defensive check
        if not isinstance(x, torch.Tensor):
            print(f"[ERROR] Non-tensor input in forward: {type(x)}")
            # Return zeros with appropriate shape
            batch_size = 1  # Default value
            return (
                torch.zeros((batch_size, 80), device=self.device),
                torch.zeros((batch_size, 1), device=self.device)
            )
            
        # Original logic
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        
        class_logits = self.classifier(x)
        ood_logits = self.ood_classifier(x)
        
        return class_logits, ood_logits

    def _get_preds(self, logits, ood_logits):
        ood_preds = torch.sigmoid(ood_logits) > self.ood_threshold
        class_preds = torch.sigmoid(logits) > self.sigmoid_thresholds.to(logits.device)
        class_preds[~ood_preds.expand(-1, class_preds.size(1))] = 0
        return class_preds

    def training_step(self, batch, batch_idx):
        if isinstance(batch, dict):  # CombinedLoader
            try:
                # Make sure we have both parts of the batch
                if "id" not in batch or "ood" not in batch:
                    print("Missing keys in batch dict:", batch.keys())
                    # Fall back to regular processing
                    x, y = next(iter(batch.values()))
                    class_logits, _ = self(x)
                    loss = self.loss_fn_class(class_logits, y)
                    return loss
                    
                id_batch = batch["id"]
                ood_batch = batch["ood"]
                
                # Extra safety checks
                if not isinstance(id_batch, (list, tuple)) or len(id_batch) < 2:
                    print(f"Invalid id_batch structure: {type(id_batch)}")
                    # Fall back
                    x, y = next(iter(batch.values()))
                    class_logits, _ = self(x)
                    loss = self.loss_fn_class(class_logits, y)
                    return loss
                    
                # Unpack ID batch
                x_id, y_class = id_batch
                
                # Final check: ensure x_id is a tensor
                if not isinstance(x_id, torch.Tensor):
                    print(f"x_id is not a tensor: {type(x_id)}")
                    # Create dummy tensor
                    x_id = torch.zeros((batch_size, 3, 224, 224), device=self.device)
                    
                class_logits, ood_logits_id = self(x_id)
                
                # Rest of your combined loader code...
            
            except Exception as e:
                print(f"Error in training_step with CombinedLoader: {e}")
                # Fall back to simpler processing
                x, y = next(iter(batch.values()))
                class_logits, _ = self(x)
                loss = self.loss_fn_class(class_logits, y)
                return loss
        else:
            # Regular batch processing...
            x, y = batch
            class_logits, _ = self(x)
            loss = self.loss_fn_class(class_logits, y)
            return loss

    def validation_step(self, batch, batch_idx):
        """
        Validation step handling both classification and OOD detection evaluation.
        Updates multiple metrics and collects data for threshold tuning.
        """
        id_batch = batch["id"]
        ood_batch = batch["ood"]

        # === Process In-Distribution Samples ===
        x_id, y_class = id_batch
        class_logits, ood_logits_id = self(x_id)

        # Convert to float for loss + metrics
        y_class = y_class.float()

        # Classification Loss
        loss_class = self.loss_fn_class(class_logits, y_class)

        # Raw predictions (no masking)
        class_preds = torch.sigmoid(class_logits) > self.sigmoid_thresholds.to(class_logits.device)
        ood_preds = torch.sigmoid(ood_logits_id).view(-1) > 0.5  # [batch_size]

        # ==== MASK OOD SAMPLES ====
        id_mask = ood_preds == 0
        class_preds_id = class_preds[id_mask]
        y_class_id = y_class[id_mask]

        # === Update metrics: ONLY ON ID SAMPLES ===
        if class_preds_id.numel() > 0:
            self.val_acc.update(class_preds_id, y_class_id)
            self.val_precision.update(class_preds_id, y_class_id)
            self.val_recall.update(class_preds_id, y_class_id)
            self.val_f1.update(class_preds_id, y_class_id)

        # === Update ID-only metrics (without OOD masking) ===
        self.val_acc_id.update(class_preds, y_class)
        self.val_f1_id.update(class_preds, y_class)
        self.val_precision_id.update(class_preds, y_class)
        self.val_recall_id.update(class_preds, y_class)

        # === Save for threshold tuning (before masking) ===
        self.validation_logits.append(class_logits.detach().cpu())
        self.validation_targets.append(y_class.detach().cpu())

        # === Process OOD samples ===
        x_ood, y_ood = ood_batch
        _, ood_logits_ood = self(x_ood)

        # OOD labels: 0 for ID, 1 for OOD
        all_ood_logits = torch.cat([
            ood_logits_id.view(-1),     # [batch_size]
            ood_logits_ood.view(-1)
        ], dim=0)

        all_ood_labels = torch.cat([
            torch.zeros_like(ood_logits_id.view(-1)),  # ID = 0
            y_ood.float().view(-1)                     # OOD = 1
        ], dim=0)

        # OOD Loss
        loss_ood = self.loss_fn_ood(all_ood_logits, all_ood_labels)

        # === Final Combined Loss ===
        loss = loss_class + 0.5 * loss_ood

        # === Log Metrics ===
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)

        # Track how many OOD samples were filtered (i.e., predicted as OOD)
        ood_filtered_count = (~id_mask).sum()
        self.log("val_ood_id_filtered", ood_filtered_count, on_epoch=True)

        return loss
    
    def test_step(self, batch, batch_idx):
        """
        Test step for final model evaluation.
        Similar to validation but with additional logging.
        Only evaluates classification metrics on in-distribution samples.
        """
        id_batch = batch["id"]
        x_id, y_class = id_batch
        class_logits, ood_logits_id = self(x_id)

        y_class = y_class.float()

        # Predict class and OOD
        class_preds = torch.sigmoid(class_logits) > self.sigmoid_thresholds.to(class_logits.device)
        ood_preds = torch.sigmoid(ood_logits_id).view(-1) > 0.5  # [batch_size]

        # Mask: only keep ID samples (OOD_pred == False)
        id_mask = ood_preds == 0
        class_preds_id = class_preds[id_mask]
        y_class_id = y_class[id_mask]

        # === Update classification metrics on ID samples only ===
        if class_preds_id.numel() > 0:
            self.val_acc.update(class_preds_id, y_class_id)
            self.val_precision.update(class_preds_id, y_class_id)
            self.val_recall.update(class_preds_id, y_class_id)
            self.val_f1.update(class_preds_id, y_class_id)

        # === Also update unmasked ID-specific metrics (for completeness/statistics) ===
        self.val_acc_id.update(class_preds, y_class)
        self.val_f1_id.update(class_preds, y_class)
        self.val_precision_id.update(class_preds, y_class)
        self.val_recall_id.update(class_preds, y_class)

        # === Log raw predictions and targets ===
        self.validation_step_y_hats.append(class_preds.cpu())
        self.validation_step_ys.append(y_class.cpu())

        return {'preds': class_preds, 'targets': y_class}

    def on_validation_epoch_end(self):
        if self.validation_logits:
            all_logits = torch.cat(self.validation_logits, dim=0)  # Ensure batch stacking
            all_targets = torch.cat(self.validation_targets, dim=0)
            new_thresholds = self._calculate_optimal_thresholds(all_logits, all_targets)

            # Use in-place assignment instead of .copy_() to avoid issues
            self.sigmoid_thresholds = (0.7 * self.sigmoid_thresholds + 0.3 * new_thresholds).detach()

            # Log the average threshold
            self.log('avg_threshold', torch.mean(self.sigmoid_thresholds).item(), prog_bar=True)

            # Clear buffers
            self.validation_logits.clear()
            self.validation_targets.clear()

        # Log metrics
        self.log('val_acc', self.val_acc.compute(), on_epoch=True, prog_bar=True)
        self.log('val_precision', self.val_precision.compute(), on_epoch=True, prog_bar=True)
        self.log('val_recall', self.val_recall.compute(), on_epoch=True, prog_bar=True)
        self.log('val_f1', self.val_f1.compute(), on_epoch=True, prog_bar=True)

        # Reset metrics
        self.val_acc.reset()
        self.val_precision.reset()
        self.val_recall.reset()
        self.val_f1.reset()
        self.ood_acc.reset()

    def _calculate_optimal_thresholds(self, logits, targets):
        """Core threshold optimization logic"""
        probs = torch.sigmoid(logits).cpu().numpy().astype(np.float32)  # Ensure float32
        targets = targets.cpu().numpy().astype(np.float32)  # Ensure float32
        thresholds = np.full(probs.shape[1], 0.5, dtype=np.float32)  # Default to 0.5 and float32
        print(f"Min prob per class: {probs.min(axis=0)}")
        print(f"Max prob per class: {probs.max(axis=0)}")

        for i in range(probs.shape[1]):

            if np.sum(targets[:, i]) == 0:
                # print(f"Warning: No positive samples for class {i}, keeping threshold at 0.5")
                continue  # Skip threshold tuning for this class

            try:
                # Precision-recall curve with F1 optimization
                precision, recall, threshs = precision_recall_curve(targets[:, i], probs[:, i])
                f1_scores = (2 * precision * recall) / (precision + recall + 1e-8)
                best_idx = np.nanargmax(f1_scores)

                if best_idx < len(threshs):
                    thresholds[i] = threshs[best_idx]
            except Exception as e:
                print(f"Error computing threshold for class {i}: {e}")

        return torch.tensor(thresholds, device=self.device, dtype=torch.float32) #return tensor

    def on_epoch_start(self):
        optimizer = self.optimizers()
        
        if isinstance(optimizer, list):  # If multiple optimizers exist
            optimizer = optimizer[0]  # Pick the first one (modify if needed)
        
        current_lr = optimizer.param_groups[0]['lr']
        self.log('lr', current_lr, on_epoch=True, prog_bar=True)


    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        
        # Ensure num_training_steps is an int
        total_training_steps = int(self.trainer.estimated_stepping_batches)  
        warmup_steps = int(0.1 * total_training_steps)  # 10% of steps for warmup

        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=warmup_steps,
            num_training_steps=total_training_steps  # Now explicitly an int
        )
        return [optimizer], [{"scheduler": scheduler, "interval": "step"}]

Best threshold: 0.3527
Best validation loss: 0.0591

## train test split

In [9]:
from dataset import CustomDataset, CustomOODDataset,target_to_oh
from torch.utils.data import ConcatDataset

def create_train_test_split_proportional(dataset, test_ratio=0.1, seed=42, transform=None):
    """
    Create a train-test split proportional to the dataset's labels.

    Args:
        dataset (ImageFolder): The dataset to split.
        test_ratio (float): Proportion of the dataset to include in the test split.
        seed (int): Random seed for reproducibility.
        transform: Image transformation for preprocessing.

    Returns:
        train_dataset, test_dataset, idx_to_label: The train/test split datasets and label mapping.
    """
    random.seed(seed)

    # Group samples by label
    label_to_samples = defaultdict(list)
    for sample in dataset.samples:
        label_to_samples[sample[1]].append(sample)

    train_samples = []
    test_samples = []

    # Split the dataset into train and test samples
    for label, samples in label_to_samples.items():
        random.shuffle(samples)
        num_test = int(len(samples) * test_ratio)
        test_samples.extend(samples[:num_test])
        train_samples.extend(samples[num_test:])

    # Create ImageFolder datasets for train and test
    train_dataset = ImageFolder(dataset.root, transform=transform)
    train_dataset.samples = train_samples
    train_dataset.targets = [sample[1] for sample in train_samples]  # Update targets

    test_dataset = ImageFolder(dataset.root, transform=transform)
    test_dataset.samples = test_samples
    test_dataset.targets = [sample[1] for sample in test_samples]  # Update targets

    # Use the class_to_idx from the original dataset for label mapping
    idx_to_label = {v: k for k, v in dataset.class_to_idx.items()}

    return train_dataset, test_dataset, idx_to_label


from dataset import CustomDataset,TensorLabelDataset,custom_collate_fn
class MyDataModule(pl.LightningDataModule):
    def __init__(
        self,
        train_data,
        val_data,
        ood_data=None,
        val_ood_data=None,
        test_dataset=None,
        batch_size=32,
        num_workers=4,
        persistent_workers=True,
        idx_to_label=None,
        collate_fn=custom_collate_fn
    ):
        super().__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.ood_data = ood_data
        self.val_ood_data = val_ood_data
        self.test_data = test_dataset
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.persistent_workers = persistent_workers
        self.idx_to_label = idx_to_label
        self.collate_fn = collate_fn
        
    def setup(self, stage=None):
        # Create proper datasets
        if isinstance(self.train_data, ConcatDataset):
            self.train_dataset = self.train_data  # Don't wrap it again
        else:
            self.train_dataset = CustomDataset(self.train_data, self.idx_to_label)
        self.val_dataset = CustomDataset(self.val_data, self.idx_to_label)

        # Set up OOD datasets if available
        if self.ood_data is not None:
            if isinstance(self.ood_data, CustomOODDataset) or hasattr(self.ood_data, '__getitem__'):
                self.train_ood_dataset = self.ood_data
            else:
                print(f"WARNING: ood_data is not a proper dataset: {type(self.ood_data)}")
                self.train_ood_dataset = None
        else:
            self.train_ood_dataset = None

        if self.val_ood_data is not None:
            if isinstance(self.val_ood_data, CustomOODDataset) or hasattr(self.val_ood_data, '__getitem__'):
                self.val_ood_dataset = self.val_ood_data
            else:
                print(f"WARNING: val_ood_data is not a proper dataset: {type(self.val_ood_data)}")
                self.val_ood_dataset = None
        else:
            self.val_ood_dataset = None

        if self.test_data:
            print("Test dataset is VALID")
            self.test_dataset = CustomDataset(self.test_data, self.idx_to_label)
        else:
            print("Test dataset is INVALID")
            self.test_dataset = None

    def train_dataloader(self):
        id_loader = DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            persistent_workers=self.persistent_workers,
            collate_fn=self.collate_fn
        )

        if self.train_ood_dataset is not None:
            ood_loader = DataLoader(
                self.train_ood_dataset,
                batch_size=self.batch_size,
                shuffle=True,
                num_workers=self.num_workers,
                persistent_workers=self.persistent_workers,
                collate_fn=self.collate_fn
            )

            combined = CombinedLoader({"id": id_loader, "ood": ood_loader}, mode="min_size")

            # ✨ Force building the iterator manually
            _ = iter(combined)

            return combined
        else:
            print("No OOD data")
            return id_loader

    def val_dataloader(self):
        id_loader = DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            persistent_workers=self.persistent_workers,
            collate_fn=self.collate_fn  # Add this
        )

        if hasattr(self, "val_ood_dataset") and self.val_ood_dataset is not None:
            ood_loader = DataLoader(
                self.val_ood_dataset,
                batch_size=self.batch_size,
                shuffle=False,
                num_workers=self.num_workers,
                persistent_workers=self.persistent_workers,
                collate_fn=self.collate_fn  # Add this
            )
            return CombinedLoader({"id": id_loader, "ood": ood_loader}, mode="min_size")
        
        return id_loader


    def test_dataloader(self):
        if self.test_dataset is None:
            print("No test dataset")
            return None
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            persistent_workers=self.persistent_workers
        )



ImportError: cannot import name 'custom_collate_fn' from 'dataset' (/Users/beszabo/Documents/meme/dataset.py)

In [10]:
transform_ = transforms.Compose([ 
    transforms.Resize((224, 224)),
    transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Apply exclusions
dataset = ImageFolder('125images', transform=None)
ood_dataset = CustomOODDataset(data_path=r"_ood",transform=transform_)
print(f"Original dataset size: {len(dataset)}")
print(f"OOD dataset size: {len(ood_dataset)}")

# Create the train-test split and update label mapping
train_dataset, test_dataset, idx_to_label = create_train_test_split_proportional(dataset, test_ratio=0.08, seed=42, transform=transform_)
print()
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}", '\n')
print("Unique class indices in dataset:", set([sample[1] for sample in dataset.samples]), '\n')
print("Number of unique classes:", len(set([sample[1] for sample in dataset.samples])), '\n')
print("Train label distribution:", Counter([sample[1] for sample in train_dataset.samples]), '\n')
print("Test label distribution:", Counter([sample[1] for sample in test_dataset.samples]), '\n')


Original dataset size: 10000
OOD dataset size: 10000

Train dataset size: 9200
Test dataset size: 800 

Unique class indices in dataset: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79} 

Number of unique classes: 80 

Train label distribution: Counter({0: 115, 1: 115, 2: 115, 3: 115, 4: 115, 5: 115, 6: 115, 7: 115, 8: 115, 9: 115, 10: 115, 11: 115, 12: 115, 13: 115, 14: 115, 15: 115, 16: 115, 17: 115, 18: 115, 19: 115, 20: 115, 21: 115, 22: 115, 23: 115, 24: 115, 25: 115, 26: 115, 27: 115, 28: 115, 29: 115, 30: 115, 31: 115, 32: 115, 33: 115, 34: 115, 35: 115, 36: 115, 37: 115, 38: 115, 39: 115, 40: 115, 41: 115, 42: 115, 43: 115, 44: 115, 45: 115, 46: 115, 47: 115, 48: 115, 49: 115, 50: 115, 51: 115, 52: 115, 53: 115, 54: 115,

## training

In [6]:
lr = 0.007323
# threshold=0.4796

batch_size= 23
dropout_rate=0.3


# szarnyprobalgatasok


In [12]:
# KFold Cross-Validation
#lr = 0.008274600824983372
#threshold = 0.3527

# or 0.3395    es batch 51
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.naive_bayes import LabelBinarizer
lb = LabelBinarizer()

kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation
val_losses = []

# Assuming you have `train_dataset` defined as an ImageFolder dataset
for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(train_dataset)) ,lb.fit_transform(np.array([train_dataset[i][1] for i in range(len(train_dataset))])))):
    print(f"Fold {fold + 1}/{kf.n_splits}")

    # Split the dataset into train and validation subsets using indices
    train_subset = Subset(train_dataset, train_idx)
    val_subset = Subset(train_dataset, val_idx)
    # In your training_step:
    # print(f"x dtype: {train_dataset[1]}")

    # Create Data Module for each fold
    data_module = MyDataModule(
        train_data=train_subset,
        val_data=val_subset,
        test_dataset=test_dataset,
        batch_size=batch_size,
        num_workers=1,
        persistent_workers=True
    )
    
    data_module.setup()
    print("Setup ran successfully")

    # Setup the model for multilabel classification
    model = ResNetWrapper(lr=lr, dropout_rate=dropout_rate)

    # Logger
    logger = TensorBoardLogger("with_ood", name=f"ood_{fold}")

    # ModelCheckpoint to save only the best models (monitor F1 score for multilabel)
    checkpoint_callback = ModelCheckpoint(
        monitor="val_f1",  # Monitor validation F1 score
        mode="max",        # Maximize the F1 score
        save_top_k=1,      # Save only the best model
        filename="{epoch}-{val_f1:.4f}"
    )

    # EarlyStopping based on F1 score
    early_stopping = EarlyStopping(
        monitor="val_f1",  # Use F1 for stopping
        patience=3,        # Stop after 3 non-improving epochs
        mode="max"
    )

    # Setup the Trainer
    trainer = Trainer(
        logger=logger,
        max_epochs=20,
        devices='auto',  # Adjust based on your hardware
        accelerator="mps",  # Use "gpu" or "tpu" based on availability
        callbacks=[early_stopping, checkpoint_callback], 
        fast_dev_run=False    
    )

    # Train the model
    trainer.fit(model, datamodule=data_module)

    # Log validation loss for this fold
    try:
        val_losses.append(trainer.callback_metrics["val_loss"].item())
    except KeyError:
        print(f"Warning: val_loss not found in callback metrics for fold {fold + 1}")
        print("Available metrics:", trainer.callback_metrics.keys())

# After cross-validation, retrieve the best model path
best_model_path = checkpoint_callback.best_model_path
print("Best model saved at:", best_model_path)


Fold 1/5
Setup ran successfully


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Loading `train_dataloader` to estimate number of stepping batches.
/Users/beszabo/Documents/meme/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.

   | Name              | Type                | Params | Mode 
-------------------------------------------------------------------
0  | feature_extractor | Sequential          | 65.3 M | train
1  | ood_classifier    | Sequential          | 2.0 K  | train
2  | classifier        | Sequential          | 165 K  | train
3  | loss_fn_class     | BCEWithLogitsLoss   | 0      | train
4  | loss_fn_ood       | BCEWithLogitsLoss   | 0      | train
5  | train_acc         | MultilabelAccuracy  | 0   

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/beszabo/Documents/meme/.venv/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  2.57it/s]Min prob per class: [0.44864318 0.47446796 0.47121352 0.4830181  0.45217678 0.46485037
 0.46559355 0.48167637 0.47121856 0.45722637 0.4750608  0.4640526
 0.44614488 0.45403677 0.4743428  0.43470535 0.454275   0.4510584
 0.47363344 0.45081773 0.4806931  0.44685844 0.46435332 0.4397898
 0.47071484 0.46782994 0.47209623 0.4542673  0.47141275 0.45619273
 0.47381327 0.4743326  0.45529974 0.48668107 0.47047064 0.46757802
 0.47793588 0.45183706 0.48534647 0.47370687 0.47350484 0.49971476
 0.46086103 0.47578844 0.45000303 0.46229392 0.484217   0.46802756
 0.48967963 0.46435407 0.45013908 0.47055012 0.46265486 0.47254327
 0.46270803 0.46008402 0.46816078 0.44910964 0.46424243 0.47493726
 0.4974303  0.48675212 0.45129684 0.47811618 0.46400458 0.47958142
 0.48791116 0.44994363 0.48161522 0.46385694 0.4613328  0.48550096
 0.44486597 0.48059264 0.4727877  0.43957534 0.47966468 0.47031322
 0.49411163 0.47127107 0.47555137]
Ma

...

Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  3.24it/s]Min prob per class: [0.4383197  0.48074412 0.4809654  0.46545428 0.4792385  0.50253403
 0.46327585 0.48101383 0.44423515 0.4865572  0.4808612  0.47394562
 0.4484189  0.4663606  0.46804366 0.46802548 0.47742763 0.45312014
 0.460666   0.4651361  0.4635306  0.45293587 0.45980546 0.47081718
 0.44973263 0.45205328 0.45396164 0.48635268 0.47552845 0.464676
 0.46436018 0.47962382 0.45093763 0.4574163  0.4460278  0.46978518
 0.46804866 0.47699013 0.43519455 0.44866112 0.45739177 0.46461383
 0.47145054 0.47006285 0.47508758 0.4395605  0.46092018 0.46738112
 0.46216184 0.4735565  0.45495236 0.4486762  0.47627437 0.46010998
 0.4737329  0.4741783  0.455088   0.45302844 0.4576917  0.481038
 0.4662836  0.47168222 0.46572495 0.44689685 0.45756426 0.48277307
 0.45790905 0.46022877 0.46917796 0.47646844 0.46423933 0.47395483
 0.48247963 0.4679949  0.4612945  0.47317323 0.4564904  0.42724732
 0.47062314 0.47340992 0.46346158]
Max

/Users/beszabo/Documents/meme/.venv/lib/python3.13/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric BinaryAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
/Users/beszabo/Documents/meme/.venv/lib/python3.13/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric MultilabelAccuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
/Users/beszabo/Documents/meme/.venv/lib/python3.13/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric MultilabelPrecision was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028
/Users/beszabo/Documents/meme/.

Epoch 0: 100%|██████████| 640/640 [03:34<00:00,  2.98it/s, v_num=0, train_loss_step=0.167] Min prob per class: [2.84465059e-05 2.88435003e-05 4.00547906e-05 3.88958215e-05
 3.19092433e-05 2.86175418e-05 5.29242607e-05 2.97304996e-05
 2.64978444e-05 2.00272898e-05 1.92738153e-05 1.51766189e-05
 3.67254906e-05 2.22720591e-05 2.73415844e-05 3.63380241e-05
 2.41254456e-05 2.54726474e-05 1.71078849e-04 3.52782481e-05
 5.75215054e-05 9.87989642e-06 2.49519017e-05 8.77369457e-05
 3.07932423e-05 2.82610436e-05 2.88765532e-05 2.75533184e-05
 2.25426920e-05 1.28833672e-05 2.09100053e-05 6.91904206e-05
 1.19478236e-05 1.26072828e-05 8.33996673e-05 4.63653196e-05
 5.11767721e-05 4.28708809e-05 5.50541881e-05 4.20774559e-05
 7.14470807e-05 1.03597245e-04 2.75499533e-05 1.94029290e-05
 2.10390808e-05 2.85414480e-05 6.71263551e-05 1.59509436e-05
 3.40504630e-05 4.73408400e-05 3.12787743e-05 6.04259621e-05
 1.08162130e-04 4.01299221e-05 3.59841106e-05 1.37878687e-05
 1.28208221e-05 1.76483609e-05 4.60

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 640/640 [04:25<00:00,  2.41it/s, v_num=0, train_loss_step=0.063, val_loss=0.123, val_ood_acc=0.922, val_acc_all=0.996, val_precision_all=0.911, val_recall_all=0.734, val_f1_all=0.813, avg_threshold=0.370, val_acc=0.996, val_precision=0.911, val_recall=0.734, val_f1=0.813, train_loss_epoch=0.121, train_acc=0.993, train_precision=0.922, train_recall=0.467, train_f1=0.620]
Best model saved at: with_ood/ood_4/version_0/checkpoints/epoch=17-val_f1=0.8157.ckpt


In [ ]:
import matplotlib.pyplot as plt
image, label = train_dataset[3]
print(idx_to_label.get(image))

plt.imshow(image.permute(1, 2, 0))  # A csatornákat (C, H, W) átrendezzük (H, W, C) formátumba
plt.title(f"Label: {label}")
plt.show()


## test 

In [ ]:
!pip install pretty-confusion-matrix
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics

In [49]:
from sklearn.metrics import precision_recall_fscore_support, multilabel_confusion_matrix

best_model_path="/kaggle/working/lightning_logs/multilabel_training_fold_0/version_5/checkpoints/epoch=22-val_loss=0.04.ckpt"
best_model_path="lightning_logs/multilabel_training_fold_4/version_10/checkpoints/epoch=17-val_f1=0.6584.ckpt"
best_model_path="/kaggle/input/may_be_the_best_resnet/pytorch/default/1/version_10/checkpoints/epoch=17-val_f1=0.6584.ckpt"
best_model_path="lightning_logs/multilabel_training_fold_4/version_0/checkpoints/epoch=22-val_f1=0.6924.ckxpt"
best_model_path="lightning_logs/multilabel_training_fold_4/version_0/checkpoints/epoch=15-val_f1=0.6850.ckpt" 
best_model_path="with_ood/ood_4/version_0/checkpoints/epoch=17-val_f1=0.8157.ckpt"
best_model_path="with_ood_80_classes_onlyID_metrics/ood_fold_0/version_4/checkpoints/epoch=13-val_f1=0.7128.ckpt"

best_model = ResNetWrapper.load_from_checkpoint(best_model_path, lr=lr, dropout_rate=0.1)

mxs = []
f1s = []
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, multilabel_confusion_matrix

import torch
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, multilabel_confusion_matrix
from torch.utils.data import DataLoader, Dataset, Subset
import lightning.pytorch as pl
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from torchmetrics.classification import MultilabelF1Score
from tqdm import tqdm
from sklearn.model_selection import KFold
def evaluate_model(model, test_loader, num_classes=80):
    """Evaluate the model with stored thresholds"""
    model.eval()
    if(test_loader==None):
        print("test_loader is None")
        return None
    device = next(model.parameters()).device
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for batch in test_loader:
            #print(len(batch))
            x, y_class= batch
            x, y_class = x.to(device), y_class.to(device)
            class_logits, ood_logits = best_model(x)
        
            #ood_probs = torch.sigmoid(ood_logits).cpu().numpy()

            # Get predictions using model's get_preds method
            preds = best_model._get_preds(class_logits, ood_logits).cpu().numpy()
            #preds = (class_probs > model.sigmoid_thresholds.to(device)).cpu().numpy()

            # Convert targets to multi-hot encoding if needed
            targets = y_class.cpu().numpy()
            if targets.ndim == 1:  # If targets are in index format instead of multi-hot
                multi_hot_targets = np.zeros((targets.shape[0], num_classes))
                for i, labels in enumerate(targets):
                    multi_hot_targets[i, labels] = 1  # Convert to one-hot
            else:
                multi_hot_targets = targets  # Already in multi-hot format

            all_preds.append(preds)
            all_targets.append(multi_hot_targets)

    # Concatenate along the first axis
    all_preds = np.concatenate(all_preds, axis=0)  # Shape (N, 80)
    all_targets = np.concatenate(all_targets, axis=0)  # Shape (N, 80)

    #print(f"Preds shape: {all_preds.shape}, Targets shape: {all_targets.shape}")

    # Ensure correct shape before computing metrics
    assert all_preds.shape == all_targets.shape, f"Shape mismatch: {all_preds.shape} vs {all_targets.shape}"

    # Calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_targets, all_preds, average=None, zero_division=0
    )

    return precision, recall, f1



if __name__ == "__main__":


    # Load dataset (ensure this is defined properly)
    data_module = MyDataModule(train_dataset, train_dataset, test_dataset, batch_size, num_workers=2, idx_to_label=idx_to_label)
    #test_dataset = data_module.test_dataloader()

    data_module.setup()

    # Load dataset and create dataloaders
    test_loader = data_module.test_dataloader()
    # Split the dataset into train and validation subsets using indices


    # Load model
    best_model = ResNetWrapper.load_from_checkpoint(
        best_model_path,
        lr=lr,  # Or the appropriate learning rate
        dropout_rate=dropout_rate
    )
    best_model.eval()

    # Run evaluation
    precision, recall, f1 = evaluate_model(best_model, test_loader)
    
    # Output metrics
    print(f"Macro F1: {np.nanmean(f1):.4f}")
    print(f"Macro Precision: {np.nanmean(precision):.4f}")
    print(f"Macro Recall: {np.nanmean(recall):.4f}")


False
Macro F1: 0.3046
Macro Precision: 0.6973
Macro Recall: 0.2112


# Deployment

In [18]:
best_model.sigmoid_thresholds

tensor([0.2849, 0.2963, 0.2818, 0.6259, 0.2959, 0.2011, 0.4135, 0.3734, 0.2627,
        0.2619, 0.1802, 0.3827, 0.5185, 0.1720, 0.3604, 0.1651, 0.2453, 0.4623,
        0.1707, 0.2890, 0.3945, 0.4414, 0.3920, 0.1822, 0.2859, 0.3029, 0.2686,
        0.2115, 0.5593, 0.4130, 0.1735, 0.5432, 0.4935, 0.4109, 0.1949, 0.2178,
        0.3441, 0.3421, 0.3135, 0.3135, 0.1489, 0.1800, 0.4276, 0.2586, 0.3230,
        0.2559, 0.2908, 0.2778, 0.6469, 0.4203, 0.3101, 0.4571, 0.4003, 0.2587,
        0.3394, 0.2790, 0.2593, 0.3019, 0.3836, 0.1453, 0.3771, 0.3427, 0.2840,
        0.4190, 0.2334, 0.6731, 0.4133, 0.3434, 0.3068, 0.2820, 0.2273, 0.2123,
        0.1689, 0.3102, 0.3898, 0.1775, 0.2295, 0.3089, 0.3682, 0.3530],
       device='mps:0')

In [22]:
# kepeket loaderbe
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

# best_model_path="finals/per_class_threshold_4/version_0/checkpoints/epoch=11-val_f1=0.6729.ckpt"
# best_model = ResNetWrapper.load_from_checkpoint(best_model_path, lr=lr, dropout_rate=dropout_rate)
best_model_path="with_ood_80_classes_onlyID_metrics/ood_fold_0/version_4/checkpoints/epoch=13-val_f1=0.7128.ckpt"

class CustomImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('png', 'jpg', 'jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image  # Mivel nincsenek címkék, csak a képet adjuk vissza

# Példa használat:
deploy_set = CustomImageDataset("/Users/beszabo/codes/arctic_shift/scripts/downloaded_images", transform=transform_)

# Test the evaluation
test_data_module = MyDataModule(None, None, deploy_set, batch_size, num_workers=2)
test_loader = test_data_module.test_dataloader()
 # Ensure model is on the appropriate device
device = torch.device("mps" if torch.cuda.is_available() else "cpu")
best_model.to(device)
best_model.eval()  # Set model to evaluation mode

all_preds = []
all_targets = []

# Iterate through test DataLoader
with torch.no_grad():
    for batch in test_loader:
        x= batch
        
        # Move data to the same device as the model
        x = x.to(device)

        logits = best_model(x)
        # Sigmoid activation + thresholding
        preds = (torch.sigmoid(logits)  > best_model.sigmoid_thresholds.to(device)).cpu().numpy()


        # Append batch predictions and targets
        all_preds.append(preds)# Preds
all_preds = np.concatenate(all_preds, axis=0)

# Convert predictions to class indices
pred_classes = [np.where(pred == 1)[0] for pred in all_preds]


True


TypeError: 'NoneType' object is not iterable

In [36]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

import multiprocessing
multiprocessing.set_start_method('fork', force=True)


best_model_path="with_ood_80_classes_onlyID_metrics/ood_fold_0/version_4/checkpoints/epoch=13-val_f1=0.7128.ckpt"


# Custom dataset for deployment images
class CustomImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('png', 'jpg', 'jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image  # Only return the image (no labels in deployment)
    
best_model = ResNetWrapper.load_from_checkpoint(best_model_path, lr=lr, dropout_rate=dropout_rate)

# Load deployment dataset
deploy_set = CustomImageDataset("/Users/beszabo/codes/arctic_shift/scripts/downloaded_images", transform=transform_)

# # Create DataLoader
# deploy_loader = DataLoader(deploy_set, batch_size=32, shuffle=False, num_workers=2)

# # Ensure model is on the appropriate device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# best_model.to(device)
# best_model.eval()  # Set model to evaluation mode

# all_preds = []

# # Iterate through test DataLoader
# with torch.no_grad():
#     for batch in deploy_loader:
#         x = batch.to(device)

#         logits = best_model(x)
#         probs = torch.sigmoid(logits)  # Convert logits to probabilities

#         # Use the model's stored sigmoid thresholds for prediction
#         preds = (probs > best_model.sigmoid_thresholds.to(device)).cpu().numpy()
        
#         all_preds.append(preds)

# # Concatenate predictions
# all_preds = np.concatenate(all_preds, axis=0)

N = 100  # Change this to the number of images you want to process

# Limit dataset to first N images
limited_dataset = torch.utils.data.Subset(deploy_set, range(min(N, len(deploy_set))))

# Create DataLoader with limited samples
limited_loader = DataLoader(limited_dataset, batch_size=32, shuffle=False, num_workers=2)

# Ensure model is on the correct device
device = torch.device("mps" if torch.cuda.is_available() else "cpu")
best_model.to(device)
best_model.eval()

all_preds = []
all_ood_scores = []  # Track OOD scores for analysis

with torch.no_grad():
    for batch in limited_loader:
        x = batch.to(device)
        
        # Get both class and OOD logits
        class_logits, ood_logits = best_model(x)
        
        # Get OOD probabilities for analysis
        ood_probs = torch.sigmoid(ood_logits).cpu().numpy()
        all_ood_scores.extend(ood_probs)
        
        # Use the model's built-in prediction method
        preds = best_model._get_preds(class_logits, ood_logits).cpu().numpy()
        all_preds.append(preds)

# Concatenate predictions
all_preds = np.concatenate(all_preds, axis=0)
all_ood_scores = np.array(all_ood_scores)

# Analyze OOD scores
print(f"Average OOD score: {np.mean(all_ood_scores):.4f}")
print(f"OOD score distribution: min={np.min(all_ood_scores):.4f}, max={np.max(all_ood_scores):.4f}")

# Count occurrences per class
class_counts = np.sum(all_preds, axis=0)

# Print class-wise image counts
for class_idx, count in enumerate(class_counts):
    if count > 0:
        print(f"Class {idx_to_label.get(class_idx)}: {count} images")

Average OOD score: 0.4969
OOD score distribution: min=0.4564, max=0.5563


In [ ]:
import os
import shutil
from torchvision.utils import save_image
from torchvision import transforms
from PIL import Image

# Define the output directory
output_dir = "/Users/beszabo/Documents/meme/preds"

# Create subdirectories for each label
for label in idx_to_label.values():
    os.makedirs(os.path.join(output_dir, label), exist_ok=True)

# Define the inverse transform (example for normalization)
inverse_transform = transforms.Compose([
    transforms.Normalize(mean=[-mean/std for mean, std in zip([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])], std=[1/std for std in [0.229, 0.224, 0.225]]),
    transforms.ToPILImage()
])

# Dictionary to count the number of images per class
class_counts = {label: 0 for label in idx_to_label.values()}

# Iterate through the predictions and save the images in the corresponding directories
for i, preds in enumerate(all_preds):
    img_path = deploy_set.image_paths[i]
    img = Image.open(img_path).convert("RGB")
    img_tensor = transform_(img)  # Apply the same transform used in the dataset

    for j, pred in enumerate(preds):
        if pred == 1:
            label = idx_to_label[j]
            save_path = os.path.join(output_dir, label, f"image_{i}.png")
            
            # Reverse the transformations
            img_reversed = inverse_transform(img_tensor)
            img_reversed.save(save_path)
            
            # Increment the class count
            class_counts[label] += 1
            print(label)

print("Images saved in the output directory.")
print("Class counts:", class_counts)

Zara images
Heineken images
Samsung images
Dior images
Oracle images
LEGO images
Airbnb images
Nestlé images
Facebook images
YouTube images
KFC images
Nescafé images
Sony images
Huawei images
Images saved in the output directory.
Class counts: {'Airbnb images': 1, 'Amazon images': 0, 'AmericanExpress images': 0, 'Apple images': 0, 'Audi images': 0, 'BMW images': 0, 'Budweiser images': 0, 'Burberry images': 0, 'Canon images': 0, 'Cartier images': 0, 'Chanel images': 0, 'Cisco images': 0, 'CocaCola images': 0, 'Colgate images': 0, 'Corona images': 0, 'DHL images': 0, 'Danone images': 0, 'Dior images': 1, 'Disney images': 0, 'Facebook images': 1, 'FedEx images': 0, 'Ferrari images': 0, 'Ford images': 0, 'Gillette images': 0, 'Google images': 0, 'Gucci images': 0, 'HM images': 0, 'HP images': 0, 'Heineken images': 1, 'Hennessy images': 0, 'Honda images': 0, 'Huawei images': 1, 'Hyundai images': 0, 'IBM images': 0, 'IKEA images': 0, 'Instagram images': 0, 'Intel images': 0, 'JackDaniels i

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming you already have the reverse index mapping: idx_to_label
mxs_0 = mxs[0]
# Calculate number of rows and columns for the subplots (80 matrices)
n_rows = 10  # You can adjust this for different grid sizes
n_cols = 8   # Adjusted to fit 80 matrices in a grid

# Set up the matplotlib figure
fig, axes = plt.subplots(n_rows, n_cols, figsize=(24, 16))  # Adjust the size as needed
axes = axes.ravel()  # Flatten the axes array to index them easily

# Plot each confusion matrix
for i, mx in enumerate(mxs_0):
    ax = axes[i]
    
    # Plot the heatmap with masked zeros
    sns.heatmap(mx, annot=True, fmt='d', vmin=0.5, vmax=1, cbar=False, ax=ax, square=True)
    
    # Set title with the index and label name
    label_name = idx_to_label.get(i, f"Label {i}")  # Get the label name or default to "Label i"
    ax.set_title(f'{label_name} (Index {i})')
    ax.set_xlabel('Predicted', fontsize=14)
    ax.set_ylabel('Actual', fontsize=14)

for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()


KeyboardInterrupt: 

## Bayesian optimization

In [7]:
from torch.utils.data import ConcatDataset, Subset
import numpy as np
import torch
from bayes_opt import BayesianOptimization
from lightning.pytorch.utilities.combined_loader import CombinedLoader
import tensorboardX, tensorboard 

def objective(sigmoid_threshold):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    val_losses = []
    

    # Alkalmazz transzformációt és kezelj multi-hot címkéket egyetlen lépésben
    for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(train_dataset)))):
        train_idx = train_idx.astype(int)
        val_idx = val_idx.astype(int)

        # Egyszerűbb adatelőkészítés - csak egyszer alkalmazzuk a transzformációt
        id_train = TensorLabelDataset(dataset=Subset(train_dataset, train_idx), transform=transform_, is_ood=False)
        id_val = TensorLabelDataset(dataset=Subset(train_dataset, val_idx), is_ood=False)

        # OOD adatok
        ood_train_indices = np.random.choice(len(ood_dataset), size=len(id_train), replace=False).astype(int)
        ood_train = TensorLabelDataset(Subset(ood_dataset, ood_train_indices), is_ood=True)

    for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(train_dataset)))):
        # print("TRAIN" + str(train_idx))
        # print(len(train_idx))
        # print(type(train_idx))
        train_idx = train_idx.astype(int)
        val_idx = val_idx.astype(int)

        # Create ID train and validation subsets
        id_train = TensorLabelDataset(Subset(train_dataset, train_idx))
        id_val = TensorLabelDataset(Subset(train_dataset, val_idx))


        # Sample OOD data for training to match length of ID train set
        ood_train_indices = np.random.choice(len(ood_dataset), size=len(id_train), replace=False).astype(int)
        ood_train = Subset(ood_dataset, ood_train_indices)

        # Sample separate OOD data for validation to match length of ID validation set
        ood_val_indices = np.random.choice(
            [i for i in range(len(ood_dataset)) if i not in ood_train_indices], 
            size=len(id_val), 
            replace=False
        ).astype(int)
        ood_val = Subset(ood_dataset, ood_val_indices)  

        # print("Sample from id_train:", id_train[0])
        # print("Sample from ood_train:", ood_train[0])

        # Combine ID and OOD for training (50-50 mix)
        combined_train = ConcatDataset([id_train, ood_train])

        # Data module with separate OOD validation data
        data_module = MyDataModule(
            combined_train, 
            id_val, 
            ood_data=ood_train,
            val_ood_data=ood_val, 
            batch_size=batch_size, 
            num_workers=1, 
            persistent_workers=True
        )
        data_module.setup()
        # Model
        model = ResNetWrapper(dropout_rate=0.3, lr=0.001, ood_threshold=sigmoid_threshold)

        logger = TensorBoardLogger("lightning_logs", name=f"threshold_finetune_fold_{fold}")
        checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1, filename="{epoch}-{val_loss:.2f}")
        early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")

        trainer = Trainer(
            logger=logger,
            max_epochs=10,
            accelerator="mps",
            callbacks=[checkpoint_callback, early_stopping]
        )

        trainer.fit(model, datamodule=data_module)

        val_loss = trainer.callback_metrics.get("val_loss", float("inf"))
        val_losses.append(val_loss.item() if isinstance(val_loss, torch.Tensor) else float(val_loss))

    return -np.mean(val_losses)  # BayesianOptimization maximizes, so return negative loss


pbounds = {
    'sigmoid_threshold': (0.3, 0.7)
} 

optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=42,
    verbose=2
) 

optimizer.maximize(init_points=3, n_iter=7)

|   iter    |  target   | sigmoi... |
-------------------------------------
Test dataset is INVALID


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Test dataset is INVALID


Loading `train_dataloader` to estimate number of stepping batches.
Traceback (most recent call last):
  File "<string>", line 1, in <module>
    from multiprocessing.spawn import spawn_main; spawn_main(tracker_fd=83, pipe_handle=97)
                                                  ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/Cellar/python@3.13/3.13.3/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'custom_collate_fn' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>


RuntimeError: DataLoader worker (pid 18589) exited unexpectedly with exit code 1. Details are lost due to multiprocessing. Rerunning with num_workers=0 may give better error trace.

In [ ]:
# Print the best parameters
best_params = optimizer.max['params']
best_params['batch_size'] = int(best_params['batch_size'])
print("Best parameters: ", best_params)
 # 0.00732   | 0.4592  val_f1= 0.461
# | 0.007323  | 0.4796    val_f1=0.528

In [ ]:
print(f"Best threshold: {optimizer.max['params']['threshold']:.4f}")
print(f"Best validation loss: {-optimizer.max['target']:.4f}")

## Download

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))
download_file('/kaggle/working/', 'out')

In [ ]:
!zip -r file.zip /kaggle/working/lightning_logs/training_proper_test
from IPython.display import FileLink
FileLink(r'file.zip')

## Testing on the labeled logo dataset

In [ ]:
!ls lightning_logs/training_proper_test/version_24/checkpoints/

In [ ]:
from torchmetrics.classification import MulticlassConfusionMatrix

metric = MulticlassConfusionMatrix(num_classes=5)
metric.update(all_labels, all_preds)
fig_, ax_ = metric.plot()

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from sklearn.metrics import f1_score
from glob import glob

# Assuming your test dataset is set up already
batch_size = 23
lr = 0.008274600824983372

# Create the test DataLoader
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Find all checkpoint files recursively across all versions
checkpoint_directory = '/kaggle/working/lightning_logs/multilabel_training_fold_2/'
checkpoint_files = glob(os.path.join(checkpoint_directory, '**', '*.ckpt'), recursive=True)

# Define a dictionary to store the performance of each checkpoint
checkpoint_performance = {}

# Iterate through each checkpoint file
for checkpoint in checkpoint_files:
    # Load the model from checkpoint
    model = best_model
    model.eval()  
    model.to('cuda')

    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    # Evaluate model
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to('cuda'), labels.to('cuda')

            # Get model predictions
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)
            _, predicted = torch.max(probabilities, 1)

            # Store predictions and labels
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Update accuracy
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    # Calculate F1 score and accuracy
    f1 = f1_score(all_labels, all_preds, average='macro')
    accuracy = 100 * correct / total

    # Store the performance metrics for the current checkpoint
    checkpoint_performance[checkpoint] = {'f1_score': f1, 'accuracy': accuracy}

    # Print results
    print(f"Checkpoint: {checkpoint}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Accuracy: {accuracy:.2f}%\n")

# Find the best checkpoint based on F1 score
best_checkpoint = max(checkpoint_performance, key=lambda x: checkpoint_performance[x]['f1_score'])
best_f1_score = checkpoint_performance[best_checkpoint]['f1_score']
best_accuracy = checkpoint_performance[best_checkpoint]['accuracy']

print("\nBest Checkpoint:")
print(f"Path: {best_checkpoint}")
print(f"F1 Score: {best_f1_score:.4f}")
print(f"Accuracy: {best_accuracy:.2f}%")


In [ ]:
import matplotlib.pyplot as plt
import torch

# Function to unnormalize images for display
def unnormalize(img_tensor, mean, std):
    img_tensor = img_tensor.clone()  # Clone the tensor to avoid modifying the original
    for t, m, s in zip(img_tensor, mean, std):
        t.mul_(s).add_(m)  # Unnormalize each channel
    return img_tensor

# Define mean and std from your transform for unnormalizing
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Create a function to plot a grid of images
def plot_image_grid(dataset, num_images=16):
    fig, axes = plt.subplots(4, 4, figsize=(8, 8))  # Grid of 4x4 for 16 images
    axes = axes.flatten()

    for i in range(num_images):
        img, label = dataset[i]
        img = unnormalize(img, mean, std)  # Unnormalize
        img = img.permute(1, 2, 0).numpy()  # Convert from CxHxW to HxWxC

        axes[i].imshow(img)
        axes[i].set_title(f"Label: {label}")
        axes[i].axis("off")

    plt.tight_layout()
    plt.show()

# Visualize some images from the training dataset
plot_image_grid(test_set, num_images=16)


In [ ]:
import os
import glob


# Define the threshold for classification
threshold = 0.8
batch_size = 23
lr = 0.008274600824983372

# Initialize the trainer
trainer = Trainer()

results = []
checkpoint = "/kaggle/input/checkpoints/lightning_logs/the training/version_4/checkpoints/epoch=8-val_loss=1.39.ckpt"

# Load the test dataset
test_dataset = ImageFolder(root='/kaggle/input/brand-logos/test', transform=transform)

# Create the test DataLoader
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Load the model from the checkpoint
model = ResNetWrapper.load_from_checkpoint(checkpoint, lr=lr, num_classes=100, 
                                            backbone_weights=models.ResNet50_Weights.IMAGENET1K_V2)
print(isinstance(model, LightningModule))
model.eval()  # Set model to evaluation mode if necessary

# Test the model using the Trainer
test_results = trainer.test(model, test_loader)

# Process the test results to compute thresholds if needed
preds = test_results[0]['preds']
labels = test_results[0]['labels']

# Convert logits to probabilities and apply thresholding
probabilities = F.softmax(preds, dim=1)
predicted = (probabilities > threshold).long().max(dim=1)[1]

# Store results
accuracy = (predicted == labels).sum().item() / len(labels)
results.append({'checkpoint': checkpoint, 'accuracy': accuracy})

# Output all results
for result in results:
    print(f"Checkpoint: {result['checkpoint']} - Accuracy: {result['accuracy']:.4f}")


In [ ]:
overall_f1_macro = f1_score(all_labels, all_preds, average='macro')   # Macro-average
print(overall_f1_macro)

### Saving best models

In [ ]:

!ls /kaggle/working/checkpoints/lightning_logs/
!ls /kaggle/working/checkpoints/lightning_logs/version_6/checkpoints
!zip -r nyertes.zip /kaggle/working/checkpoints/lightning_logs/version_13/checkpoints/epoch=14-step=2190.ckpt


In [ ]:
!zip -r version_5.zip /kaggle/working/checkpoints/lightning_logs/

In [ ]:
!pip install -U ipywidgets
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension


In [ ]:
!rm -rf /kaggle/working/*

## Test on unlabeled marketing memes

In [ ]:
import os
from PIL import Image
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
# Define your brand names in the order they appear in your list
brand_names = [
    "3m", "axa", "accenture", "adobe", "airbnb", "allianz", "amazon", "americanexpress", 
    "apple", "audi", "bmw", "budweiser", "burberry", "canon", "cartier", "caterpillar", 
    "chanel", "cisco", "citi bank", "cocacola", "colgate", "corona", "dhl", "danone", 
    "dior", "disney", "facebook", "fedex", "ferrari", "ford", "ge", "gillette", 
    "goldmansachs", "google", "gucci", "hm", "hp", "hsbc", "heineken", "hennessy", 
    "hermès", "hewlettpackardenterprise", "honda", "huawei", "hyundai", "ibm", "ikea", 
    "instagram", "intel", "jpmorgan", "jackdaniels", "johnsonjohnson", "kfc", "kelloggs", 
    "kia", "lego", "loréalparis", "linkedin", "louisvuitton", "mastercard", "mcdonalds", 
    "mercedesbenz", "microsoft", "morganstanley", "nescafé", "nespresso", "nestlé", 
    "netflix", "nike", "nintendo", "nissan", "oracle", "pampers", "panasonic", "paypal", 
    "pepsi", "philips", "porsche", "prada", "redbull", "sap", "salesforce", "samsung", 
    "santander", "sephora", "siemens", "sony", "spotify", "starbucks", "tesla", 
    "tiffanyco", "toyota", "ups", "visa", "volkswagen", "xiaomi", "youtube", "zara", 
    "adidas", "ebay"
]

class UnlabeledImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.lower().endswith(('png', 'jpg', 'jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_path


# Load the model checkpoint
num_classes = 100  # Adjust based on your actual number of classes
model = ResNetWrapper.load_from_checkpoint(checkpoint,lr=lr, num_classes=num_classes, backbone_weights=models.ResNet50_Weights.IMAGENET1K_V2)

# Move the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet50 typically takes 224x224 input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define the prediction function
def classify_images(model, image_folder, transform, device):
    dataset = UnlabeledImageDataset(image_folder, transform=transform)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
    results = {}  # Ensure 'results' dictionary is initialized

    with torch.no_grad():
        for images, img_paths in dataloader:
            images = images.to(device)
            logits = model(images)
            probs = F.softmax(logits, dim=1)  # Convert logits to probabilities
            confidences, preds = torch.max(probs, dim=1)  # Get the highest probability and its index for each prediction

            for confidence, pred, img_path in zip(confidences, preds, img_paths):
                if confidence.item() >= 0.9:  # Only consider predictions with confidence >= 80%
                    brand = brand_names[pred.item()].lower()  # Map index to brand name

                    if brand not in results:
                        results[brand] = []
                    results[brand].append(os.path.basename(img_path))  # Append the filename, not the full path

    return results

# Define the folder containing images to classify
image_folder = "/kaggle/input/5memes-for-top-100-most-valuable-brand-2023"

# Classify images and store the results in a dictionary
dict1 = classify_images(model, image_folder, transform, device)


# Classify images and store the results in a dictionary
dict1 = classify_images(model, image_folder, transform, device)
dict2 = {'apple': ['Xiaomi memes_9daf00f7-8607-4ff6-b8e1-1dd5c755f43a.jpeg', 'Apple memes_iphone-chatgpt.jpg', 'Apple memes_Image_2.jpg', 'Huawei memes_Image_3.jpg'], 'microsoft': ['Microsoft memes_Image_5.jpg', 'Microsoft memes_Image_1.jpeg', 'Microsoft memes_Image_4.jpeg', 'HP memes_190d521ac62eaf40a74e50ab39635231.jpg'], 'amazon': ['Amazon memes_Image_1.jpg', 'Amazon memes_Image_2.jpg', 'Amazon memes_Image_4.jpeg', 'Amazon memes_Image_5.jpg', 'Amazon memes_Image_3.jpg'], 'google': ['Google memes_Image_2.jpeg', 'Google memes_Image_1.jpg', 'Xiaomi memes_Image_2.jpeg', 'Google memes_Image_4.jpg', 'Google memes_Image_3.jpg'], 'samsung': ['Samsung memes_Image_1.jpeg', 'Samsung memes_Image_2.jpg', 'Samsung memes_Image_3.jpg', 'Samsung memes_Image_4.jpg', 'Samsung memes_Image_5.jpeg'], 'toyota': ['Toyota memes_Image_1.jpg', 'Toyota memes_Image_2.jpg', 'Toyota memes_Image_4.jpg'], 'mercedes-benz': ['MercedesBenz memes_Image_5.jpg', 'MercedesBenz memes_Image_3.jpg'], 'coca-cola': ['CocaCola memes_Image_2.jpeg', 'Pepsi memes_Image_3.jpg', 'CocaCola memes_Image_1.jpg'], 'nike': [], 'bmw': ['BMW memes_ayNdM3q_460s.jpg', 'BMW memes_Image_4.jpg', 'Audi memes_Image_5.jpg', 'MercedesBenz memes_Image_1.jpg'], "mcdonald's": ['McDonalds memes_Image_2.jpg', 'McDonalds memes_Image_1.jpg', 'McDonalds memes_Image_2 (copy 1).jpg'], 'tesla': ['Tesla memes_Image_2.jpg', 'Tesla memes_teslamemes-191110030713-thumbnail.webp', 'Caterpillar memes_image-1.webp', 'Tesla memes_Image_5.jpg'], 'disney': ['Disney memes_Image_1.png', 'Disney memes_Image_2.jpg', 'Disney memes_relatable-disney-memes.png'], 'louis vuitton': ['LouisVuitton memes_Image_1.jpg', 'LouisVuitton memes_tgeycyn3z4k91.jpg', 'LouisVuitton memes_0a2163e267c3e9c175e7c08bf8253318.jpg', 'LouisVuitton memes_Image_2.jpg'], 'cisco': ['Cisco memes_d77c72918e7440d37fc3eabc307ad8ac3a8025383ea34cdb6e1df27c047628f6_1.jpg', 'Cisco memes_Image_3.jpg', 'Cisco memes_Image_1.jpg', 'Cisco memes_fc0j9uamxd371.png'], 'instagram': ['Instagram memes_Image_4.jpg', 'Instagram memes_ig.jpg', 'Heineken memes_Image_1.jpg', 'Instagram memes_Image_5.jpg', 'Airbnb memes_Image_1.jpg', 'Instagram memes_person-women-on-instagram-be-like-got-new-shoes.jpeg', 'Instagram memes_Image_2.jpg', 'HewlettPackardEnterprise memes_EPo6d7mUYAAHsU0.jpg'], 'adobe': ['Adobe memes_Image_3.jpeg', 'Adobe memes_Image_1.jpeg', 'Adobe memes_Image_2.jpeg'], 'ibm': ['IBM memes_Image_2.jpg', 'IBM memes_Image_1.jpg', 'IBM memes_Image_3.jpg'], 'oracle': ['Oracle memes_EXT38P1WkAAjs_5.jpg', 'Oracle memes_Image_5.jpg', 'Oracle memes_programmerhumor-io-databases-memes-backend-memes-ce04a4d894b6035-608x613.webp'], 'sap': ['SAP memes_Image_1.jpeg', 'SAP memes_Image_3.jpg', 'SAP memes_Image_5.png', 'SAP memes_Image_2.jpeg', 'SAP memes_Image_4.jpg'], 'facebook': ['Instagram memes_Image_4.jpg', 'Heineken memes_Image_1.jpg', 'Caterpillar memes_Bj9v-r7IUAApml9.png', 'Facebook memes_Image_1.jpg', 'Facebook memes_Screenshot2021-11-01at16.29.20.jpg', 'TiffanyCo memes_98facae07556bfaab7cb6672371ff06c.jpg', 'Facebook memes_f.jpg', 'Facebook memes_Image_4.jpg', 'HewlettPackardEnterprise memes_EPo6d7mUYAAHsU0.jpg'], 'chanel': ['Chanel memes_images (1).jpeg', 'Chanel memes_Screen-Shot-2021-04-29-at-8.webp', 'Chanel memes_Image_2.jpg', 'Chanel memes_images.jpeg'], 'hermes': ['Herms memes_434053634_18427064311043375_1898595488137274113_n.jpg', 'Herms memes_Fd_gt0bUUAAeCRh.jpg'], 'intel': ['Intel memes_Image_4 (copy 1).jpeg', 'Intel memes_Image_2.jpg', 'Intel memes_Image_4.jpeg', 'Intel memes_Image_1.jpg'], 'youtube': ['YouTube memes_Image_2.jpg', 'YouTube memes_Image_3.jpg'], 'j.p. morgan': ['JPMorgan memes_xq0z4545oqe41.webp', 'MorganStanley memes_Image_1.jpg'], 'honda': ['Honda memes_Image_4.jpg', 'Honda memes_Image_5.jpg'], 'american express': ['AmericanExpress memes_Image_3.jpg', 'AmericanExpress memes_Image_4.jpg', 'AmericanExpress memes_Image_1.jpg'], 'ikea': ['IKEA memes_Image_3.jpg', 'IKEA memes_Image_5.png', 'IKEA memes_Image_2.jpg', 'IKEA memes_Image_4.jpg'], 'accenture': ['Accenture memes_Image_4.jpg', 'Accenture memes_Image_1.png', 'Accenture memes_Image_2.jpg', 'Accenture memes_Image_5.jpg'], 'allianz': ['Allianz memes_amLOMzj_460s.jpg', 'Allianz memes_Image_2.jpg', 'Allianz memes_Image_1.jpg', 'Allianz memes_images.jpeg'], 'hyundai': ['Hyundai memes_Image_3.jpeg', 'Hyundai memes_Image_4.jpg'], 'ups': ['Cisco memes_d77c72918e7440d37fc3eabc307ad8ac3a8025383ea34cdb6e1df27c047628f6_1.jpg', 'UPS memes_Image_4.jpg', 'Ferrari memes_Image_2.jpg', 'DHL memes_Image_2.jpeg', 'UPS memes_Image_3.jpeg', 'UPS memes_Image_2.jpeg'], 'gucci': ['Gucci memes_Image_3.jpg', 'Gucci memes_Image_1.jpg', 'Gucci memes_Image_2 (copy 1).jpg', 'Gucci memes_Image_2.jpg'], 'pepsi': ['Pepsi memes_Image_2.jpeg', 'Pepsi memes_Image_3.jpg', 'Pepsi memes_Image_5.jpeg'], 'sony': ['Sony memes_Image_2.jpg', 'Sony memes_Image_3.jpeg', 'Sony memes_Image_2 (copy 1).jpg', 'Sony memes_Image_1.jpeg'], 'visa': ['Visa memes_Image_3.jpg', 'AmericanExpress memes_Image_2.jpg', 'Visa memes_Image_2.jpg', 'Visa memes_Image_5.jpg', 'Visa memes_Image_4.jpg'], 'salesforce': ['Salesforce memes_Image_1.jpeg', 'Salesforce memes_Image_3.jpeg', 'Salesforce memes_Image_5.jpeg', 'Salesforce memes_Image_2.jpeg', 'Salesforce memes_Image_4.jpg'], 'netflix': ['Netflix memes_Image_4.jpg', 'Cisco memes_d77c72918e7440d37fc3eabc307ad8ac3a8025383ea34cdb6e1df27c047628f6_1.jpg', 'Netflix memes_Image_5.jpg', 'Netflix memes_Image_1.jpg'], 'paypal': ['PayPal memes_Image_4.jpeg', 'PayPal memes_Image_2.jpg', 'PayPal memes_Image_3.jpeg', 'PayPal memes_Image_1.jpeg'], 'mastercard': ['Mastercard memes_Image_2.jpg', 'Mastercard memes_Image_4.jpg', 'Mastercard memes_Image_1.jpeg', 'AmericanExpress memes_Image_2.jpg', 'Visa memes_Image_5.jpg'], 'adidas': ['adidas memes_Image_1.jpeg'], 'zara': ['Zara memes_Image_1.jpeg', 'Zara memes_Image_3.jpeg', 'Zara memes_Image_5.jpg'], 'axa': ['AXA memes_gettyimages-1226469012-612x612.jpg'], 'audi': ['Audi memes_Image_5.jpg', 'MercedesBenz memes_Image_1.jpg', 'Audi memes_Image_4.jpg'], 'airbnb': ['Airbnb memes_Image_3.png', 'Airbnb memes_Image_1.jpg', 'Airbnb memes_Image_4.jpeg', 'Airbnb memes_Image_2.jpg', 'Airbnb memes_P4BwmWt.jpg'], 'porsche': ['Porsche memes_Image_2.jpeg', 'Porsche memes_Image_1.jpg', 'Porsche memes_Image_4.jpg'], 'starbucks': ['Starbucks memes_Image_4.jpg', 'Starbucks memes_Image_2.png', 'Starbucks memes_4e59a310d37f67cbb82bcd067ee4e0c7.jpg', 'Starbucks memes_Image_3.png'], 'ge': ['GE memes_images (2).jpeg'], 'volkswagen': ['Volkswagen memes_Image_3.jpg', 'Volkswagen memes_Image_1.jpeg'], 'ford': ['Ford memes_Image_5.jpeg'], 'nescafé': [], 'siemens': ['Siemens memes_Image_5.jpg', 'Siemens memes_siemens-heh-heh-heh-v0-yf2fbabx814b1.webp', 'Siemens memes_Image_4.jpg', 'Siemens memes_A6Db4nQCQAEw2Xg.jpg'], 'goldman sachs': ['GoldmanSachs memes_Image_5.jpg', 'GoldmanSachs memes_Image_2.jpg', 'GoldmanSachs memes_Image_3.jpg', 'GoldmanSachs memes_Image_4.jpg'], 'pampers': ['Pampers memes_Image_3.jpg', 'Pampers memes_Image_5 (copy 1).jpg', 'Pampers memes_Image_5.jpg'], 'h&m': ['HM memes_tandem-x-visuals-FZOOxR2auVI-unsplash-1313x900.webp', 'HM memes_Image_4.jpeg'], 'l’oréal paris': ['LOralParis memes_Image_3.jpg', 'LOralParis memes_Image_1.jpg', 'LOralParis memes_Image_2.jpeg'], 'citi': ['Cisco memes_d77c72918e7440d37fc3eabc307ad8ac3a8025383ea34cdb6e1df27c047628f6_1.jpg', 'Citi Bank memes_Image_4.jpg', 'MorganStanley memes_Image_3.jpg', 'Citi Bank memes_citi.jpg', 'Citi Bank memes_Image_2.jpg', 'Citi Bank memes_Image_5.jpg'], 'lego': ['LEGO memes_Image_5.jpg', 'LEGO memes_images.jpeg', 'LEGO memes_Image_4.jpeg'], 'red bull': ['JackDaniels memes_Image_2.jpg', 'RedBull memes_Image_5.jpg', 'RedBull memes_Image_2.jpg', 'RedBull memes_Image_4.jpeg', 'RedBull memes_Image_3.jpg'], 'budweiser': ['Budweiser memes_Image_3.jpg', 'Budweiser memes_aoKL853_460s.jpg', 'Budweiser memes_Image_2.jpg', 'Budweiser memes_Image_4.jpg', 'Budweiser memes_Image_1.jpg'], 'ebay': ['eBay memes_Image_4.jpg', 'eBay memes_Image_2.jpg'], 'nissan': ['Nissan memes_Image_2.jpeg', 'Nissan memes_Image_4.jpg', 'Nissan memes_Image_3.jpeg', 'Nissan memes_Image_3.jpg'], 'hp': ['HewlettPackardEnterprise memes_k8m3z.jpg', 'HP memes_38ef2b77327f4020f04dc14dc41f1538.jpg', 'HP memes_190d521ac62eaf40a74e50ab39635231.jpg'], 'hsbc': ['HSBC memes_Image_1.jpg', 'HSBC memes_Image_3.jpg', 'HSBC memes_images.jpeg'], 'morgan stanley': ['MorganStanley memes_4521ccf691de4bacef41ccbb143387b2.jpg', 'MorganStanley memes_Image_4.jpg', 'MorganStanley memes_images.jpeg', 'MorganStanley memes_Image_1.jpg'], 'nestle': ['Nestl memes_Image_1.jpeg', 'Nestl memes_Image_2.jpeg', 'Nestl memes_Image_3.jpeg', 'Nestl memes_Image_4.jpeg', 'Nestl memes_Image_5.jpg'], 'philips': ['Philips memes_Image_4.jpeg', 'Philips memes_images.jpeg', 'Philips memes_Philips-Innovationandyou-1002x1417.jpg', 'Philips memes_unnamed.jpg'], 'spotify': ['Spotify memes_spotify-now-listen-most-boring-pop-charts-yes-hey-xx-xx-tap-banner-now-last-ad-just-two-songs-ago.png', 'Spotify memes_Image_5.jpeg'], 'ferrari': ['Ferrari memes_Image_5.jpg', 'Ferrari memes_Image_1.jpg', 'Ferrari memes_Image_1.jpeg', 'Ferrari memes_Image_2.jpg', 'Ferrari memes_Image_3.png'], 'nintendo': ['Philips memes_Image_4.jpeg', 'Nintendo memes_Image_5.jpeg', 'Nintendo memes_Image_4.jpeg', 'Panasonic memes_Image_3.jpeg'], 'gillette': ['Gillette memes_Image_1.jpeg', 'Gillette memes_Image_2.jpg', 'Gillette memes_Image_4.jpeg', 'Gillette memes_Image_5.jpg'], 'colgate': ['Colgate memes_Image_4.jpeg', 'Colgate memes_Image_2.jpg', 'LOralParis memes_Image_2.jpeg'], 'cartier': ['Cartier memes_CaGpc-1VIAAOpWc.png', 'Cartier memes_Image_3.jpg', 'Cartier memes_Image_1.jpeg', 'Cartier memes_65c080cea63c4.jpeg'], '3m': ['3M memes_Image_2.jpg', '3M memes_Image_5.jpg', '3M memes_Image_3.jpg', '3M memes_khkn86l1vgs41.webp'], 'dior': ['Dior memes_Image_3.jpg', 'Dior memes_Image_5.jpg', 'Dior memes_Image_2.jpg', 'Dior memes_Image_4.jpeg', 'Dior memes_Image_4.jpg'], 'santander': ['Santander memes_Image_2.jpg', 'Santander memes_Image_3.jpg', 'Santander memes_Image_4.jpg', 'Santander memes_meme1.webp'], 'danone': ['Danone memes_Image_2.jpg', 'Danone memes_Image_3.jpg', 'Danone memes_5lmocm.jpg', 'Danone memes_5b1d46878de83.jpeg', 'Danone memes_page_1.webp'], "kellogg's": ['Kelloggs memes_Image_3.jpeg', 'Kelloggs memes_Image_3.jpg'], 'linkedin': ['LinkedIn memes_Image_2.jpg'], 'corona': ['Corona memes_om9ysstu92k41.jpg'], 'fedex': ['FedEx memes_Image_1.jpg', 'FedEx memes_Image_5.jpg', 'FedEx memes_Image_2.jpeg', 'FedEx memes_Image_3.jpg'], 'caterpillar': ['Caterpillar memes_image-1.webp'], 'dhl': ['DHL memes_Image_5.jpg', 'DHL memes_Image_4.png', 'DHL memes_Image_1.jpg', 'DHL memes_Image_2.jpeg'], "jack daniel's": ['JackDaniels memes_Image_4.jpg'], 'prada': ['Prada memes_highxtar-prada-ss21-campaign-4.jpg', 'Prada memes_highxtar-prada-ss21-campaign-1.jpg', 'Prada memes_DIETPRADA3.webp', 'Prada memes_highxtar-prada-ss21-campaign-2.jpg'], 'xiaomi': ['Xiaomi memes_9daf00f7-8607-4ff6-b8e1-1dd5c755f43a.jpeg', 'Xiaomi memes_Image_4.jpeg', 'Xiaomi memes_Image_1.jpeg'], 'kia': [], 'tiffany & co.': [], 'panasonic': ['Panasonic memes_Image_5.jpg', 'Panasonic memes_Image_4.jpeg', 'Panasonic memes_Image_1.jpeg'], 'hewlett packard enterprise': ['hp enterprise', 'hewlett packard', 'HewlettPackardEnterprise memes_maxresdefault.jpg', 'HewlettPackardEnterprise memes_EPo6d7mUYAAHsU0.jpg'], 'huawei': ['Huawei memes_Image_5.jpg', 'Huawei memes_u8978vigrj031.webp', 'Huawei memes_Image_3.jpg'], 'hennessy': ['Hennessy memes_Image_4.jpg', 'Hennessy memes_Image_3.jpg', 'Hennessy memes_Image_2.jpg', 'Hennessy memes_Image_1.jpg', 'Hennessy memes_Image_5.jpg'], 'burberry': ['Burberry memes_images.jpeg', 'Burberry memes_Image_3.jpg', 'Burberry memes_4febf2b9add2cf73f8e3bcca6f67c904.jpg', 'Burberry memes_Image_5.jpg'], 'kfc': ['KFC memes_Image_4.jpg', 'KFC memes_Image_1.jpeg', 'KFC memes_Image_5.jpeg', 'KFC memes_Image_2 (copy 1).jpg'], 'johnson & johnson': ['JohnsonJohnson memes_jj-vaccine-covid-19-vaccine-memes-covid-19-memes-funny-memes-memes-twitter-memes-funny-tweets.jpeg', 'JohnsonJohnson memes_memes-for-use-to-spam-j-j-v0-2kmhfpfrrmbb1.webp', 'JohnsonJohnson memes_Image_1.jpg'], 'sephora': ['Sephora memes_Image_5.png', 'Sephora memes_Image_2.jpg', 'Sephora memes_Image_1.jpg', 'Sephora memes_Image_4.jpeg', 'Sephora memes_Image_3.jpg'], 'nespresso': ['Nespresso memes_Image_3.jpg', 'Nespresso memes_Image_1.jpg', 'Nespresso memes_Image_2.jpg', 'Nespresso memes_Image_4.jpg', 'Nespresso memes_Image_5.jpg'], 'heineken': ['Corona memes_images.jpeg', 'Heineken memes_Image_1.jpg', 'Heineken memes_Image_5.jpg'], 'canon': ['Canon memes_55bc7be8031ed.jpeg', 'Canon memes_7ugwbf.jpg', 'Canon memes_Image_1.jpg', 'Canon memes_xzgpcaalz69a1.jpg']}
result = {}

for key in dict1:
    if key in dict2:
        if key not in result:
            result[key] = dict1[key] + dict2[key]
    else:
        result[key] = dict1[key]

for key in dict2:
    if key not in result:
        result[key] = dict2[key]
print("finished")

In [ ]:
for i, (key, value_list) in enumerate(dict1.items()):
    print(f"{i} \t {key}: {value_list}\n")

In [ ]:
result

Dagsub connection, authentication

In [ ]:
!rm -r /kaggle/working/

In [ ]:
!rm -fr /kaggle/working/DIRPATH/model-epoch=14-val_loss=1.60.ckpt